**steps**
1. identify row with headers
2. map headers to apppriate column number
3. read the line above (a) component till total/sub-total
4. remove all rows with null (expect for yeild)
5. add to full_data


In [27]:
import numpy as np 
import pandas as pd

In [28]:
def read_excel_file(file_path):
    file_ext = file_path.split(".")[-1].lower()
    if file_ext == "xlsb":
        try:
            return pd.read_excel(file_path, sheet_name=None, engine="pyxlsb", dtype=str)
        except Exception as e:
            print(f"❌ Error reading {file_path} (XLSB format): {e}")
            return None
    elif file_ext in ["xls", "xlsx"]:
        try:
            return pd.read_excel(file_path, sheet_name=None, dtype=str)
        except Exception as e:
            print(f"❌ Error reading {file_path} (XLS/XLSX format): {e}")
            return None
    else:
        print(f"⚠️ Unsupported file format: {file_path}")
        return None


**test - get_scheme_name**

In [60]:
import os
path = r"data\\data\\Edelweiss Mutual Fund"
tables = os.listdir(path)

In [61]:
all_sheets = []
for filepath in tables:
    df_raw = read_excel_file(f"{path}/{filepath}")
    if df_raw is None:
        continue
    for sheet_name, sheet_df in df_raw.items():
        # if sheet_name not in sheets_to_avoid:
        all_sheets.append(sheet_df)

In [67]:
import re
re.sub("portfolio\s*\w*\s*of","","PORTFOLIO STATEMENT OF EDELWEISS CRISIL IBX 50:50 GILT PLUS SDL SEP 2028 INDEX FUND AS ON FEBRUARY 7, ".lower())

' edelweiss crisil ibx 50:50 gilt plus sdl sep 2028 index fund as on february 7, '

In [63]:
df = all_sheets[3]
df.head()

,"PORTFOLIO STATEMENT OF EDELWEISS CRISIL IBX 50:50 GILT PLUS SDL SEP 2028 INDEX FUND AS ON FEBRUARY 7, 2025",Unnamed: 1,Unnamed: 2
0,(An open-ended target maturity Index Fund inve...,NaN,NaN
1,NaN,NaN,NaN
2,Name of the Instrument,Rating/Industry,% to Net Assets
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [40]:
all_sheets[2].iloc[:,6].to_list()

[nan,
 nan,
 '% to Net Assets',
 nan,
 nan,
 nan,
 nan,
 '0.0813',
 '0.0811',
 '0.081',
 '0.0793',
 '0.0785',
 '0.0553',
 '0.0548',
 '0.0546',
 '0.0526',
 '0.0525',
 '0.0276',
 '0.0264',
 '0.0132',
 '0.7382',
 nan,
 nan,
 '0.1077',
 '0.08',
 '0.1877',
 nan,
 nan,
 '0.0685',
 '0.0685',
 nan,
 nan,
 '0.0033',
 '0.0033',
 nan,
 nan,
 '0.0023',
 '0.0023',
 nan,
 '1',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'Interim Distribution received (Rs.in lakhs)',
 '1459.663\n738.784',
 '12.03\n5.01',
 '9.19\n3.82',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [5]:
from core.amcparser import AMCPortfolioParser
t = AMCPortfolioParser(config={"ISIN_file":"ISIN/fundisin.xlsx"})

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vaibh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [77]:
df = all_sheets[1]
rows = df.fillna(" ").agg(" ".join , axis = 1).apply(str.lower)
df = df.iloc[rows[rows.apply(lambda x : "stock exchang" not in x or not ("index" in x and "stock" in x))].index.to_list()]
isin_col = 1
# row_num = t._get_investment_type(df , 12 , isin_col)
# row_num

In [78]:
df.head(15)

,Back to Index,Motilal Oswal Nifty 50 ETF,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,"Portfolio as on February 28, 2025",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sr. No.,Name of Instrument,NaN,ISIN,Rating / Industry,Quantity,Market value (Rs. In lakhs),% to Net Assets,NaN,NaN,Sector / Rating,Percent
3,NaN,Equity & Equity related,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Banks,0.30893573826710136
4,NaN,(a) Listed / awaiting listing on Stock Exchanges,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IT - Software,0.13133776142554932
5,1,HDFC Bank Limited,NaN,INE040A01034,Banks,35741,619.18,0.13271822707666778,NaN,NaN,Petroleum Products,0.08670368150510772
6,2,ICICI Bank Limited,NaN,INE090A01021,Banks,33164,399.33,0.08559436213217626,NaN,NaN,Automobiles,0.07225480934053317
7,3,Reliance Industries Limited,NaN,INE002A01018,Petroleum Products,31907,382.92,0.08207655226544884,NaN,NaN,Diversified FMCG,0.05658956743881611
8,4,Infosys Limited,NaN,INE009A01021,IT - Software,16930,285.73,0.061244614514922156,NaN,NaN,Finance,0.04306799307144677
9,5,Bharti Airtel Limited,NaN,INE397D01024,Telecom - Services,12606,197.94,0.042427551839496216,NaN,NaN,Telecom - Services,0.042427551839496216


In [7]:
df[row_num:row_num+1].fillna("").agg("".join, axis = 1)

10    Mutual Fund Units
dtype: object

In [9]:
import re
re.sub("\s+"," ","Hello world    nice game")

'Hello world nice game'

In [61]:
import re
type_name_idx = row_num 

type_name =  df[type_name_idx:type_name_idx+1].fillna("").agg(" ".join , axis = 1).iloc[0]
if("(" not in type_name): type_name = re.sub(r"[^\)]\)","",type_name)
type_name = re.sub(r"\([^)]*\)", "", type_name)
type_name = re.sub(r"[^a-zA-Z\s\&\-/\\]" , "" , type_name)
type_name = re.sub(r"(?<!\w)(nan)+(?!\w)", "", type_name ,flags=re.IGNORECASE)
type_name

' Mutual Fund Units          '

In [ ]:
rows[rows.apply(lambda x : "grand total" in x)].index.to_list()[0]

int

In [55]:
for (index ,row ) in df.iloc[2:5].iterrows():
    print(type(index))

<class 'int'>
<class 'int'>
<class 'int'>


In [ ]:
look = t.isin_lookup
def get_fund_isin(fund_name):
        import re 
        import pandas as pd
        import nltk
        nltk.download("stopwords")
        from nltk.corpus import stopwords
        tstopwords = {}
        removestopwords = lambda x : x #" ".join([word for word in x.lower().split(" ") if word not in tstopwords])
        transform = lambda x : re.sub(r"[^a-zA-z0-9]","",x).lower()

        fund_name = removestopwords(fund_name)
        fund_name = transform(fund_name).split("fund")[0]
        

        fund_names = pd.Series(look.keys()).astype(str)

        mask = fund_names.apply(lambda x : "iciciprudentialallsea" in transform(removestopwords(x)))
        candidate_fund_names = fund_names[mask].to_list()

        print(fund_name)
        print(candidate_fund_names)

        if len(candidate_fund_names):
            return look.get(candidate_fund_names[0])
        else:
            return None
print(get_fund_isin("ICICI Prudential All Seasons Bond Fund"))

iciciprudentialallseasonsbond
['icici prudential all seasons bond fund']
INF109K01GM1


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vaibh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import re 
re.sub(r"(?<!\w)(nan)+(?!\w)",""," i001 -dnang nan nannan ", flags=re.IGNORECASE)

' i001 -dnang   '

In [ ]:
"0".isdigit()

True

In [229]:
df = all_sheets[0]
df.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Sr No.,Fund Name,Fund Code
3,NaN,NaN,1,Motilal Oswal Large and Midcap Fund,YO20
4,NaN,NaN,2,Motilal Oswal Midcap Fund (Formerly known as M...,YO07
5,NaN,NaN,3,Motilal Oswal Small Cap Fund,YO46
6,NaN,NaN,4,Motilal Oswal Focused Fund (Formerly known as ...,YO05
7,NaN,NaN,5,Motilal Oswal Flexi Cap Fund,YO08
8,NaN,NaN,6,Motilal Oswal Large Cap Fund,YO47
9,NaN,NaN,7,Motilal Oswal ELSS Tax Saver Fund (Formerly Kn...,YO09


In [239]:
groww = lambda x : re.sub(r"^IB\w*-","", x)
for idx , df in enumerate(all_sheets):
    try:
        print(idx , " -- " , groww(list(filter(lambda x : "unnamed" not in x.lower() and "groww" in x.lower() , list(df.columns)))[0].strip()))
    except Exception as e:
        print("error")
        continue

0  --  Groww Large Cap Fund
1  --  Groww Liquid Fund
2  --  Groww Aggressive Hybrid Fund
3  --  Groww Dynamic Bond Fund
4  --  Groww Short Duration Fund
5  --  Groww ELSS Tax Saver Fund
6  --  Groww Value Fund
7  --  Groww Overnight Fund
8  --  Groww Nifty Total Market Index Fund
9  --  Groww Banking & Financial Services Fund
10  --  Groww Nifty Smallcap 250 Index Fund
11  --  Groww Nifty Non-Cycl Consumer Index Fund
12  --  Groww Nifty EV & New Age Auto ETF FOF
13  --  Groww Nifty EV & New Age Automotive ETF
14  --  GROWW NIFTY 1D Rate Liquid ETF
15  --  Groww Nifty India Defence ETF
16  --  Groww Nifty India Defence ETF FOF
17  --  GROWW Gold ETF
18  --  Groww Gold ETF FOF
19  --  Groww Multicap Fund
error
error


**test over**

In [ ]:
all_sheets = []

df_raw = read_excel_file(path)
if df_raw is None:
    pass
for sheet_name, sheet_df in df_raw.items():
    # if sheet_name not in sheets_to_avoid:
    all_sheets.append(sheet_df)

        

In [ ]:
df = all_sheets[0]
df = df.dropna(how='all')
df.reset_index(drop=True , inplace = True)
rows = df.fillna(" ").agg("".join , axis = 1)
df = df.iloc[rows[rows.apply(lambda x : "listing on stock exchange" not in x.lower())].index.to_list()]
df.reset_index(drop=True , inplace = True)

In [ ]:
for df in all_sheets:
    try:
        print(list(filter(lambda x : "unnamed" not in x.lower() and "icici" in x.lower() , df.iloc[0,:].astype(str).to_list())))
    except Exception as e:
        print(e)
        continue



['ICICI Prudential Balanced Advantage Fund']


In [ ]:
df.iloc[0,:].to_list()

[nan,
 'ICICI Prudential Balanced Advantage Fund',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [ ]:
def fetch_header_row(df :pd.DataFrame) -> list[str]: 
    rows = df.astype(str).agg(' '.join, axis=1)
    idx = rows[rows.apply(lambda x: "instrument" in x.lower())].index.tolist()[0]
    header_row = df.iloc[idx,:].fillna("NULL")
    header_row = [(header_row.iloc[col]) for col in range(header_row.shape[0])]
    return header_row

In [ ]:
header_row = fetch_header_row(df)
header_row

['NULL',
 'ISIN',
 'Coupon (%)',
 'Name Of the Instrument',
 'Industry+ /Rating',
 'Quantity',
 'Market/ Fair Value (Rs. in Lacs.)',
 '% to NAV',
 'Yield',
 '~YTC (AT1/Tier 2 bonds)',
 'Derivative\n% to NAV',
 'Unhedged\n% to NAV']

In [ ]:
while "NULL" in header_row:
    start = None
    for i in range(len(header_row)):
        if start == None and header_row[i] != "NULL":
            start = i
            break

    for i in range(start+1 , len(header_row)):
        if header_row[i] != "NULL":
            end = i
            break
    alter1 = df.iloc[:,start:end].fillna("").agg("".join,axis = 1)
    alter2 = df.drop(df.columns[start:end],axis = 1)
    df = pd.concat([alter1 , alter2] , axis = 1)

    header_row = fetch_header_row(df)

            
header_row

['ISIN',
 'Coupon (%)',
 'Name Of the Instrument',
 'Industry+ /Rating',
 'Quantity',
 'Market/ Fair Value (Rs. in Lacs.)',
 '% to NAV',
 'Yield',
 '~YTC (AT1/Tier 2 bonds)',
 'Derivative\n% to NAV',
 'Unhedged\n% to NAV']

## **Step 2** 

In [ ]:
base_headers = ["Name of Instrument","ISIN" , "Industry" , "Yield" , "Quantity" , "Market Value" , "Net Asset Value (NAV)"]

In [ ]:
import requests
def generate_embedding(text:str) -> list[float]:
    url = "https://lamhieu-lightweight-embeddings.hf.space/v1/embeddings"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    data = {
        "model": "snowflake-arctic-embed-l-v2.0",
        "input": text
    }

    response = requests.post(url, headers=headers, json=data)
    if response.ok:
        return response.json()["data"][0]["embedding"]
    else:
        raise Exception("No response")

In [ ]:
test_header_row=["Company/Issuer/Instrument Name", "Industry/Rating","Quantity","ISIN","Gibbrish" , "Exposure/Market Value", "yeild to Instrument","% to Nav", "Cupon"]

In [ ]:
def header_mapper(header_row , * , base_headers = ["Name of Instrument","ISIN" , "Industry" , "Yield" , "Quantity" , "Market Value" , "Net Asset Value (NAV)"]) -> {str:int}:
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity

    header_map = dict()

    base_embeddings = np.array([generate_embedding(value) for value in base_headers])
    header_row_embeddings = np.array([generate_embedding(value) for value in header_row])
    # Compute cosine similarity (shape: 5 x 10)
    similarity_matrix = cosine_similarity(base_embeddings, header_row_embeddings)

    # For each base vector, find the index of the most similar header
    most_similar_indices = np.argmax(similarity_matrix, axis=1)

    # Optionally, get the similarity score too
    most_similar_scores = np.max(similarity_matrix, axis=1)

    # Print results
    for i, (idx, score) in enumerate(zip(most_similar_indices, most_similar_scores)):
        print(f"Base vector {i} ie {base_headers[i]} is most similar to header {idx} ie {header_row[idx]} with score {score:.4f}")
        header_map[base_headers[i]] = int(idx)
        
    return header_map

In [ ]:
# header_mapper(test_header_row)

In [ ]:
header_map = header_mapper(header_row)
header_map

Base vector 0 ie Name of Instrument is most similar to header 2 ie Name Of the Instrument with score 0.9527
Base vector 1 ie ISIN is most similar to header 0 ie ISIN with score 1.0000
Base vector 2 ie Industry is most similar to header 3 ie Industry+ /Rating with score 0.6091
Base vector 3 ie Yield is most similar to header 7 ie Yield with score 1.0000
Base vector 4 ie Quantity is most similar to header 4 ie Quantity with score 1.0000
Base vector 5 ie Market Value is most similar to header 5 ie Market/ Fair Value (Rs. in Lacs.) with score 0.5944
Base vector 6 ie Net Asset Value (NAV) is most similar to header 6 ie % to NAV with score 0.5839


{'Name of Instrument': 2,
 'ISIN': 0,
 'Industry': 3,
 'Yield': 7,
 'Quantity': 4,
 'Market Value': 5,
 'Net Asset Value (NAV)': 6}

## **step 3**

## **step 4** read all lines in valid ranges

In [ ]:
import pandas as pd

def check_isin(val):
    s = str(val).lower().strip()
    s = re.sub("[^a-zA-Z0-9]" , "" , s)
    return s.startswith("in") and s[-1] in "0123456789"

def get_valid_periods(df , header_map):

    mask = df.iloc[:, header_map["ISIN"]].apply(check_isin).values

    # Find continuous True periods
    periods = []
    start = None

    for i, val in enumerate(mask):
        if val:
            if start is None:
                start = i
        else:
            if start is not None:
                periods.append((start, i - 1))
                start = None

    # Edge case: last element was True
    if start is not None:
        periods.append((start, len(mask) - 1))

    print("Passing periods:", periods)
    return periods

periods = get_valid_periods(df , header_map)

Passing periods: [(4, 58), (63, 65), (70, 71)]


In [ ]:
df

,0,"HDFC Flexi Cap Fund (An open ended equity scheme investing across large cap, mid cap & small cap stocks).2","HDFC Flexi Cap Fund (An open ended equity scheme investing across large cap, mid cap & small cap stocks).3","HDFC Flexi Cap Fund (An open ended equity scheme investing across large cap, mid cap & small cap stocks).4","HDFC Flexi Cap Fund (An open ended equity scheme investing across large cap, mid cap & small cap stocks).5","HDFC Flexi Cap Fund (An open ended equity scheme investing across large cap, mid cap & small cap stocks).6","HDFC Flexi Cap Fund (An open ended equity scheme investing across large cap, mid cap & small cap stocks).7","HDFC Flexi Cap Fund (An open ended equity scheme investing across large cap, mid cap & small cap stocks).8","HDFC Flexi Cap Fund (An open ended equity scheme investing across large cap, mid cap & small cap stocks).9",Income,Hybrid
0,Portfolio as on 31-Jan-2025Portfolio as on 31-...,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,NaN,NaN
1,ISIN,Coupon (%),Name Of the Instrument,Industry+ /Rating,Quantity,Market/ Fair Value (Rs. in Lacs.),% to NAV,Yield,~YTC (AT1/Tier 2 bonds),Derivative\n% to NAV,Unhedged\n% to NAV
2,EQUITY & EQUITY RELATED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,INE090A01021|,NaN,ICICI Bank Ltd.,Banks,49500000,620136,9.4,NaN,NaN,0,9.4
...,...,...,...,...,...,...,...,...,...,...,...
137,7) Total value and percentage of Illiquid Equi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,8) IDCW stands for Income Distribution cum Cap...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,9) Riskometer based on Scheme Portfolio and Po...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,Scheme Riskometer:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
full_data = pd.DataFrame(columns= list(header_map.keys()) + ["type" , "scheme" , "amc name"])

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
for (start_idx , end_idx) in periods:
    scheme_name = re.findall(r'\b[a-zA-Z-\\/\s]+\b', df[start_idx-1:start_idx].fillna("").agg("".join , axis = 1).iloc[0])[0]
    print("\n",scheme_name)

    for (index , row) in df.iloc[start_idx:end_idx+1].iterrows():
        values = header_map.copy()
        for (key , idx) in header_map.items():
            values[key] = row.iloc[idx]
        print(f"{index} ",end=" , ") # just to keep track

        full_data = pd.concat([full_data , pd.DataFrame([values])],ignore_index=True)
print("sheet over")



 Equity
4  , 5  , 6  , 7  , 8  , 9  , 10  , 11  , 12  , 13  , 14  , 15  , 16  , 17  , 18  , 19  , 20  , 21  , 22  , 23  , 24  , 25  , 26  , 27  , 28  , 29  , 30  , 31  , 32  , 33  , 34  , 35  , 36  , 37  , 38  , 39  , 40  , 41  , 42  , 43  , 44  , 45  , 46  , 47  , 48  , 49  , 50  , 51  , 52  , 53  , 54  , 55  , 56  , 57  , 58  , 
 Government Securities
63  , 64  , 65  , 
 Units issued by ReIT
70  , 71  , sheet over


In [ ]:
full_data = full_data.drop_duplicates()

In [ ]:
full_data

,Name of Instrument,ISIN,Industry,Yield,Quantity,Market Value,Net Asset Value (NAV),type,scheme,amc name
0,ICICI Bank Ltd.,INE090A01021|,Banks,NaN,49500000,620136,9.4,NaN,NaN,NaN
1,HDFC Bank Ltd.£,INE040A01034|,Banks,NaN,36000000,611550,9.27,NaN,NaN,NaN
2,Axis Bank Ltd.,INE238A01034|,Banks,NaN,56000000,552216,8.37,NaN,NaN,NaN
3,Kotak Mahindra Bank Limited,INE237A01028|,Banks,NaN,16500000,313714.5,4.76,NaN,NaN,NaN
4,Maruti Suzuki India Limited,INE585B01010|,Automobiles,NaN,2500000,307766.25,4.67,NaN,NaN,NaN
5,SBI Life Insurance Company Ltd.,INE123W01016|,Insurance,NaN,19500000,289302,4.39,NaN,NaN,NaN
6,Cipla Ltd.,INE059A01026|,Pharmaceuticals & Biotechnology,NaN,19200000,284044.8,4.31,NaN,NaN,NaN
7,Bharti Airtel Ltd.,INE397D01024|,Telecom - Services,NaN,16200000,263460.6,3.99,NaN,NaN,NaN
8,HCL Technologies Ltd.,INE860A01027|,IT - Software,NaN,12000000,207054,3.14,NaN,NaN,NaN
9,PIRAMAL PHARMA LTD,INE0DK501011|,Pharmaceuticals & Biotechnology,NaN,75597677,176066.99,2.67,NaN,NaN,NaN


In [ ]:
class A:
    def one(self):
        print("Hello")
        return 1


In [ ]:
aa = A()
def two(a,b):
    print(f"{a+b} two")
    return 2
aa.one = two

In [ ]:
aa.one(1,2)

3 two


2

In [ ]:
header_row

['ISIN',
 'Coupon (%)',
 'Name Of the Instrument',
 'Industry+ /Rating',
 'Quantity',
 'Market/ Fair Value (Rs. in Lacs.)',
 '% to NAV',
 'Yield',
 '~YTC (AT1/Tier 2 bonds)',
 'Derivative\n% to NAV',
 'Unhedged\n% to NAV']

In [ ]:
start = 0
for i in range(len(header_row)):
    if header_row[i] != "NULL":
        end = i
        alter = df.iloc[:,start:end].fillna("").agg("".join,axis = 1)
        alter2 = df.drop(df.columns[start:end],axis = 1)
        pd.concat([alter , alter2] , axis = 1 )
        start = i

In [49]:
list.get = lambda x : self[x] or None

TypeError: cannot set 'get' attribute of immutable type 'list'

In [ ]:
get_valid_periods(df , header_map)

Passing periods: [(14, 58), (63, 65), (70, 71)]


[(14, 58), (63, 65), (70, 71)]

In [118]:
import yaml
f = open("config/amc_configs2.yaml")
data = yaml.safe_load(f)
f.close()
data.get("Defaudlts",{}).get("OutputDirectory")

In [18]:

import os
import pandas as pd
path = "./cleaned" 
files = [f"{path}/{file}" for file in list(os.walk(path))[0][2]]

In [19]:
file = files[2]

In [26]:
df = pd.read_excel(file)
df["yield"].to_list()

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 8.0,
 7.4,
 7.37,
 7.68,
 7.63,
 6.52,
 6.4,
 6.51,
 7.7,
 7.75,
 7.28,
 7.71,
 7.32,
 7.26,
 7.28,
 7.28,
 7.3,
 7.28,
 7.25,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 8.15,
 7.05,
 7.03,
 7.26,
 0.0,
 0.0,
 7.65,
 7.87,
 7.63,
 7.52,
 7.05,
 7.02,
 6.6,
 7.75,
 7.3,
 0.0,
 0.0,
 0.0,
 7.4,
 7.78,
 7.49,
 7.82,
 7.35,
 7.52,
 7.87,
 7.65,
 6.74,
 6.6,
 7.63,
 7.66,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,

In [246]:
numeric_columns = ["coupon" , "quantity" , "market value (mkt) (lakh)" , "% to net assets (nav)" , "yield", "yield to call (ytc)"]

In [247]:
for col in numeric_columns:
    if df[col].dtype == "object":
        print(col)

In [8]:
import numpy as np

In [13]:
np.divide(100,0)

C:\Users\vaibh\AppData\Local\Temp\ipykernel_16312\1301769774.py:1: RuntimeWarning: divide by zero encountered in divide
  np.divide(100,0)


np.float64(inf)

In [146]:
import re
str(re.match("\d+[^a-z.]|\d+.\d+[^.\w]" , "1.22").string)

AttributeError: 'NoneType' object has no attribute 'string'

In [215]:
floatFilter = lambda x : re.findall("\d+\.{1}\d+",x)
integerFilter = lambda x : re.findall("\d+",x)

In [217]:
integerFilter("aab377g4.5")

['377', '4', '5']

In [70]:
path= r"C:\\Users\\vaibh\\Documents\\IPYNB_notebooks\\interview\\final_cleaned\\Kotak Mutual Fund.xlsx"
df = pd.read_excel(path)
df.head()


,name of instrument,isin,coupon,industry / rating,quantity,market value (mkt) ( rs lakh ),% to net assets (nav),yield,yield to call (ytc),Type,Scheme Name,AMC,Scheme ISIN
0,Inter Globe Aviation Ltd,INE646L01027,0,Transport Services,11079.0,479.09,0.2007,0.0,0,Equity & Equity related,Kotak Nifty India Tourism Index Fund,Kotak Mutual Fund,INF174KA1TD6
1,INDIAN HOTELS CO LTD,INE053A01029,0,Leisure Services,60289.0,461.03,0.1931,0.0,0,Equity & Equity related,Kotak Nifty India Tourism Index Fund,Kotak Mutual Fund,INF174KA1TD6
2,Jubilant Foodworks Limited,INE797F01020,0,Leisure Services,43712.0,307.69,0.1289,0.0,0,Equity & Equity related,Kotak Nifty India Tourism Index Fund,Kotak Mutual Fund,INF174KA1TD6
3,GMR AIRPORTS LIMITED,INE776C01039,0,Transport Infrastructure,410633.0,298.41,0.1250,0.0,0,Equity & Equity related,Kotak Nifty India Tourism Index Fund,Kotak Mutual Fund,INF174KA1TD6
4,Indian Railway Catering And Tourism Corpora...,INE335Y01020,0,Leisure Services,34539.0,284.01,0.1190,0.0,0,Equity & Equity related,Kotak Nifty India Tourism Index Fund,Kotak Mutual Fund,INF174KA1TD6
